In [36]:
# https://mne.tools/stable/auto_tutorials/forward/20_source_alignment.html

import numpy as np
import nibabel as nib
from scipy import linalg

import mne
from mne.io.constants import FIFF

data_path = mne.datasets.sample.data_path()
# subjects_dir = data_path / "subjects"
subjects_dir = 'C:/Users/zhouz/mne_data/MNE-fsaverage-data'
raw_fname = data_path / "MEG" / "sample" / "sample_audvis_raw.fif"
raw = mne.io.read_raw_fif(raw_fname)
# trans_fname = data_path / "MEG" / "sample" / "sample_audvis_raw-trans.fif"
# trans_fname = "/home/zhibinz2/Documents/GitHub/MEG_EEG_Source_Localization/sample_fsaverage_trans.fif"
# trans_fname = 'C:/Users/zhouz/GitHub/MEG_EEG_Source_Localization/sample_fsaverage_trans.fif'
# trans_fname = 'C:/Users/zhouz/GitHub/MEG_EEG_Source_Localization/sample_fsaverage_manual_trans.fif'
trans_fname = 'sample_fsaverage_manual_trans.fif'
# trans_fname = mne.read_trans(trans_fname)
trans = mne.read_trans(trans_fname)
# trans = "fsaverage" # mne.read_trans(trans_fname)

# src = mne.read_source_spaces(subjects_dir / "fsaverage" / "bem" / "fsaverage-ico-5-src.fif")
src = mne.read_source_spaces('C:/Users/zhouz/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif')

# Load the T1 file and change the header information to the correct units
# t1w = nib.load(data_path / "subjects" / "fsaverage" / "mri" / "T1.mgz")
t1w = nib.load('C:/Users/zhouz/mne_data/MNE-fsaverage-data/fsaverage/mri/T1.mgz')
t1w = nib.Nifti1Image(t1w.dataobj, t1w.affine)
t1w.header["xyzt_units"] = np.array(10, dtype="uint8")
t1_mgh = nib.MGHImage(t1w.dataobj, t1w.affine)

# fiducials = 'C:/Users/zhouz/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-fiducials.fif'

Opening raw data file C:\Users\zhouz\mne_data\MNE-sample-data\MEG\sample\sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read


In [30]:
raw.info

<Info | 21 non-empty values
 acq_pars: ACQch001 110113 ACQch002 110112 ACQch003 110111 ACQch004 110122 ...
 bads: 2 items (MEG 2443, EEG 053)
 ch_names: MEG 0113, MEG 0112, MEG 0111, MEG 0122, MEG 0123, MEG 0121, MEG ...
 chs: 204 Gradiometers, 102 Magnetometers, 9 Stimulus, 60 EEG, 1 EOG
 custom_ref_applied: False
 description: acquisition (megacq) VectorView system at NMR-MGH
 dev_head_t: MEG device -> head transform
 dig: 146 items (3 Cardinal, 4 HPI, 61 EEG, 78 Extra)
 events: 1 item (list)
 experimenter: MEG
 file_id: 4 items (dict)
 highpass: 0.1 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 lowpass: 172.2 Hz
 meas_date: 2002-12-03 19:01:10 UTC
 meas_id: 4 items (dict)
 nchan: 376
 proj_id: 1 item (ndarray)
 proj_name: test
 projs: PCA-v1: off, PCA-v2: off, PCA-v3: off
 sfreq: 600.6 Hz
>

In [28]:
# The HPI coils are shown as magenta spheres. Coregistration happens at the beginning of the recording and 
# the head↔meg transformation matrix is stored in
raw.info['dev_head_t']

<Transform | MEG device->head>
[[ 0.99141997 -0.03993636 -0.12446729 -0.00612931]
 [ 0.06066115  0.98401171  0.16745624  0.00006361]
 [ 0.11578966 -0.17356974  0.97799116  0.06474152]
 [ 0.          0.          0.          1.        ]]

In [37]:
# Understanding coordinate frames
fig = mne.viz.plot_alignment(
    raw.info,
    trans=trans,
    subject="fsaverage",
    subjects_dir=subjects_dir,
    surfaces="head",
    show_axes=True,
    dig=True,
    eeg=[],
    meg="sensors",
    coord_frame="meg",
    mri_fiducials="estimated",
)
# mri_fiducials= fiducials
# mri_fiducials="estimated"
# mri_fiducials=True,

mne.viz.set_3d_view(fig, 90, 90, distance=0.6, focalpoint=(0.0, 0.0, 0.0))
print(
    "Distance from head origin to MEG origin: %0.1f mm"
    % (1000 * np.linalg.norm(raw.info["dev_head_t"]["trans"][:3, 3]))
)
# print(
#     "Distance from head origin to MRI origin: %0.1f mm"
#     % (1000 * np.linalg.norm(trans["trans"][:3, 3]))
# )
dists = mne.dig_mri_distances(raw.info, trans, "fsaverage", subjects_dir=subjects_dir)
print(
    "Distance from %s digitized points to head surface: %0.1f mm"
    % (len(dists), 1000 * np.mean(dists))
)

Using outer_skin.surf for head surface.
Channel types::	grad: 203, mag: 102
Distance from head origin to MEG origin: 65.0 mm
Using surface from C:\Users\zhouz\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-head-dense.fif.
Distance from 72 digitized points to head surface: 4.8 mm


In [35]:
# good example
mne.viz.plot_alignment(
    raw.info,
    trans=trans,
    subject="fsaverage",
    src=src,
    subjects_dir=subjects_dir,
    dig=True,
    surfaces=['head',"pial"],
    coord_frame="mri",
    meg='sensors'
)

# surfaces=['head",white"]
# surfaces=['head-dense",white"]
# surfaces=['inner_skull',"white"]
# surfaces=['inner_skull',"pial"],
# meg='sensors','ref','helmet'

Using outer_skin.surf for head surface.
Channel types::	grad: 203, mag: 102, eeg: 59


In [38]:
# The head surface is stored in "mri" coordinate frame
# (origin at center of volume, units=mm)
seghead_rr, seghead_tri = mne.read_surface(
    subjects_dir / "fsaverage" / "surf" / "lh.seghead"
)

# To put the scalp in the "head" coordinate frame, we apply the inverse of
# the precomputed `trans` (which maps head → mri)
# mri_to_head = linalg.inv(trans["trans"])
mri_to_head = linalg.inv(trans["trans"])
scalp_pts_in_head_coord = mne.transforms.apply_trans(mri_to_head, seghead_rr, move=True)

# To put the scalp in the "meg" coordinate frame, we use the inverse of
# raw.info['dev_head_t']
head_to_meg = linalg.inv(raw.info["dev_head_t"]["trans"])
scalp_pts_in_meg_coord = mne.transforms.apply_trans(
    head_to_meg, scalp_pts_in_head_coord, move=True
)

# The "mri_voxel"→"mri" transform is embedded in the header of the T1 image
# file. We'll invert it and then apply it to the original `seghead_rr` points.
# No unit conversion necessary: this transform expects mm and the scalp surface
# is defined in mm.
vox_to_mri = t1_mgh.header.get_vox2ras_tkr()
mri_to_vox = linalg.inv(vox_to_mri)
scalp_points_in_vox = mne.transforms.apply_trans(mri_to_vox, seghead_rr, move=True)

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [17]:
def add_head(renderer, points, color, opacity=0.95):
    renderer.mesh(*points.T, triangles=seghead_tri, color=color, opacity=opacity)


renderer = mne.viz.backends.renderer.create_3d_figure(
    size=(600, 600), bgcolor="w", scene=False
)
add_head(renderer, seghead_rr, "gray")
add_head(renderer, scalp_pts_in_meg_coord, "blue")
add_head(renderer, scalp_pts_in_head_coord, "pink")
add_head(renderer, scalp_points_in_vox, "green")
mne.viz.set_3d_view(
    figure=renderer.figure,
    distance=800,
    focalpoint=(0.0, 30.0, 30.0),
    elevation=105,
    azimuth=180,
)
renderer.show()

NameError: name 'scalp_pts_in_meg_coord' is not defined

In [18]:
# Get the nasion:
nasion = [
    p
    for p in raw.info["dig"]
    if p["kind"] == FIFF.FIFFV_POINT_CARDINAL and p["ident"] == FIFF.FIFFV_POINT_NASION
][0]
assert nasion["coord_frame"] == FIFF.FIFFV_COORD_HEAD
nasion = nasion["r"]  # get just the XYZ values

# Transform it from head to MRI space (recall that `trans` is head → mri)
nasion_mri = mne.transforms.apply_trans(trans, nasion, move=True)
# Then transform to voxel space, after converting from meters to millimeters
nasion_vox = mne.transforms.apply_trans(mri_to_vox, nasion_mri * 1e3, move=True)
# Plot it to make sure the transforms worked
renderer = mne.viz.backends.renderer.create_3d_figure(
    size=(400, 400), bgcolor="w", scene=False
)
add_head(renderer, scalp_points_in_vox, "green", opacity=1)
renderer.sphere(center=nasion_vox, color="orange", scale=10)
mne.viz.set_3d_view(
    figure=renderer.figure,
    distance=600.0,
    focalpoint=(0.0, 125.0, 250.0),
    elevation=45,
    azimuth=180,
)
renderer.show()


TypeError: string indices must be integers